In [12]:
import pandas as pd 
import urllib3
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from datetime import datetime
from tqdm import tqdm
from datetime import date
import itertools

import warnings
warnings = 0

In [13]:
def convert_date(d):
    try:
        date = datetime.strptime(d, '%Y %b %d').strftime('%Y-%m-%d')
    except:
        date = datetime.strptime(d, '%Y %b').strftime('%Y-%m')
    return date

In [14]:
def clean_carriages(array):
    try:
        array = [line.replace('\r', '') for line in array]
    except:
        print('no return carriages')
    
    try:
        array = [line.replace('\n', '') for line in array]
    except:
        print('no newline carriages')
    
    return(array)

# might be able to use soup.stripped_strings:

In [15]:
def get_PI(masterList, grantNumber):
    a = []
    b = []
    temp = masterList[['Base_Project.1','PI']]

    for i in range(len(temp['Base_Project.1'])):
        if temp['Base_Project.1'][i] not in a:
            a.append(temp['Base_Project.1'][i])
            b.append(temp['PI'][i])

    dic = dict(zip(a,b))

    return([*map(dic.get,grantNumber)])

    

In [16]:
def get_df_info(pubmed_url, ids, grant_names_1):
    url = pubmed_url + ids
    
    
    page = urlopen(url)
    html_doc = page.read()
    html = html_doc.decode("utf-8")

    # parse html content
    soup = BeautifulSoup( html_doc , 'html.parser')
    # =============================================== #
    # preprint or not:
    if (len(re.findall('preprint', str(soup))) < 1):
        print_type = ''
    else:
        print_type = 'preprint'
    
    # =============================================== #
    # Citation (since im lazy)
    try:
        citation = soup.find("div", {"class": "article-citation"}).text.replace('\r', '').replace('\n', '')
        citation = re.findall("(^.*?)(?:doi)", citation)[0].strip()
    except:
        citation = ''
    
    # =============================================== #
    # Finding by class name (title)
    title = soup.find(class_ = "heading-title")
    try:
        title = str(clean_carriages(title)[0]).strip()
    except:
        title = str(title)
        title = temp.replace('\n', '').replace('<h1 class="heading-title">','').replace('</h1>','')
        try:
            title = temp.replace('<i>', '').replace('</i>','').strip()

        except:
            print("no italics")
    
    # =============================================== #
    # Finding by class name (authors)
    author = soup.find_all(class_ = "authors-list-item")
    author_name = re.findall("data-ga-label=\"(.+?)\"", str(author))
    
    # =============================================== #
    # Finding date of publication (cit)
    DOP = soup.find(class_ = "cit").string
    try:
        date = convert_date(DOP.split(';')[0])
    except:
        date = convert_date(DOP.split('.')[0])
    
    # =============================================== #
    # Finding the correct grant
    # parse html content
    soup = BeautifulSoup( html_doc , 'html.parser')
    grant = soup.find_all(class_ = "grant-item")
    grant = re.findall("data-ga-label=\"(.+?)\"", str(grant))

    str_grants = ', '.join(grant)
    # print(str_grants)
    
    # =============================================== #
    # clean out grants that are not in the right format:
    # bgrants = re.findall('(\w{3} \d{2}X\d{3})|(CA\d{6})', ' '.join(grant))
    bgrants = re.findall('(\w{3} CA\d{6})|(\w{3} \d{2}X\d{3})|(CA\d{6})', ' '.join(grant))
    out = list(filter(None,itertools.chain(*bgrants)))
    
    for i in out:
        base_grant = ''
        seroNet_grant = ''

        try:
            if i.split(' ')[1] in grant_names_1:
                base_grant = i.replace(' ','')
                seroNet_grant = i.split(' ')[1]
        except:
            base_grant = ''
            seroNet_grant = i
    
    # Finding publishing journal 
    soup = BeautifulSoup(html_doc, 'lxml', from_encoding='utf-8')
    journal = re.findall('(?:meta content=\")(.*?)(?:name=\"citation_journal_title)',str(soup))[0][:-2]
    
    
    return title, author_name, journal, date, seroNet_grant, base_grant, print_type, citation
    # # Finding by class name (title)
    # title = soup.find(class_ = "heading-title")
    # title = str(clean_carriages(title)[0]).strip()

## Pull Papers ##

In [17]:
pmids_url = 'https://pubmed.ncbi.nlm.nih.gov/?term=CA260508+OR+CA260476+OR+CA260539+OR+CA' \
'60584+OR+CA260507+OR+CA260469+OR+CA260526+OR+CA260588+OR+CA260513+OR+CA260541+OR+CA260462'\
'+OR+CA261276+OR+CA261277+OR+CA260582+OR+CA260543+OR+CA260492+OR+CA260591+OR+CA260581+OR+' \
'CA260517+OR+CA260563+OR+CA260560+OR+21X089+OR+21X090+OR+21X091+OR+21X092%5BGrant+Number%5'\
'D&format=pubmed&sort=date&size=200'

master_list = pd.ExcelFile('/Users/liualg/National Institutes of Health/NCI-FNL SeroNet'\
                            ' Team - Curation channel/Master List Publications_2021-PMIDS and authors.xlsx')

grant_names = re.findall('(CA\d{6})', pmids_url)
grant_names += re.findall('(\d{2}X\d{3})', pmids_url)

In [18]:
PMID_input = '34767812 34861167'
PMID_list = PMID_input.split(' ')
search_url = 'https://pubmed.ncbi.nlm.nih.gov/'

In [19]:
PMID = []
title_list = []
author_list = []
journal_list = []
date_list = []
grant_list = []
base_list = []
paperType_list = []
paper_citation = []
curator = []

for ids in tqdm(PMID_list):
    title, author_name, journal, date_str, seroNet_grant, base_grant, print_type, citation = get_df_info(search_url, ids, grant_names)
    
    PMID.append(ids)
    title_list.append(title)
    author_list.append(str(author_name))
    journal_list.append(journal)
    date_list.append(date_str)
    grant_list.append(seroNet_grant)
    base_list.append(base_grant)
    paperType_list.append(print_type)
    paper_citation.append(citation)
    
    
grantCuratorDic = dict(zip(master_list.parse(sheet_name= 'curatorGrantMapping')['Grant'], 
                           master_list.parse(sheet_name= 'curatorGrantMapping')['primaryCurator']))

curator = [*map(grantCuratorDic.get,grant_list)]



100%|██████████| 2/2 [00:01<00:00,  1.33it/s]


In [20]:
empty = ['']*len(PMID)
updating_fields = {
    'Publication Owner':curator,
    'JIRA Epic #':empty,
    'Base_Project':base_list,
    'Status/ImmPort Study ID':empty,
    'Base_Project.1':grant_list, 
    'PMID':PMID,
    'PMID_N':['PMID_'+str(x) for x in PMID],
    'PMID_link':empty,
    'Publication_Title':title_list, 
    'Journal':journal_list,
    'Author(s)':author_list,
    'Pub Date':date_list,
    'PI':empty,
    'print_type': paperType_list,
    'citation': paper_citation
}

In [21]:
temp = pd.DataFrame(updating_fields)

temp['Author(s)'] = temp['Author(s)'].str.replace('\'','')
temp['Author(s)'] = temp['Author(s)'].str.replace('[','')
temp['Author(s)'] = temp['Author(s)'].str.replace(']','')

temp['PI'] = get_PI(master_list.parse(sheet_name= 'Full Publication List'), temp['Base_Project.1'])
today = date.today()
today = today.strftime("%y-%m-%d")
temp.to_csv(f'./new_pmids/data_pull_{today}.csv')

temp.head()

<ipython-input-21-9cb5486a651e>:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  temp['Author(s)'] = temp['Author(s)'].str.replace('[','')
<ipython-input-21-9cb5486a651e>:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  temp['Author(s)'] = temp['Author(s)'].str.replace(']','')
/Users/liualg/opt/anaconda3/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Publication Owner,JIRA Epic #,Base_Project,Status/ImmPort Study ID,Base_Project.1,PMID,PMID_N,PMID_link,Publication_Title,Journal,Author(s),Pub Date,PI,print_type,citation
0,Liu,,U54CA260492,,CA260492,34767812,PMID_34767812,,Progression and Resolution of Severe Acute Res...,The American journal of pathology,"Kathleen R Mulka, Sarah E Beck, Clarisse V Sol...",2021-11-10,"Klein, Sabra L",,Am J Pathol. 2021 Nov 10;S0002-9440(21)00472-7.
1,Liu,,U54CA260517,,CA260517,34861167,PMID_34861167,,Direct comparison of antibody responses to fou...,Cell host &amp; microbe,"Naranjargal J Dashdorj, Oliver F Wirz, Kathari...",2021-12-08,"Boyd, Scott D",,Cell Host Microbe. 2021 Dec 8;29(12):1738-1743...


In [55]:
for i,o in enumerate(temp.PMID):
    print(f"CURATOR: {temp['Publication Owner'][i]}")
    print(f"Title: {temp['Publication_Title'][i]}")
    print(f"Journal: {temp['Journal'][i]}")
    print(f"Pubmed: https://pubmed.ncbi.nlm.nih.gov/{temp['PMID'][i]}")
    print(temp['PMID_N'][i])
    print(temp['PI'][i])
    print(temp['Base_Project.1'][i])
    print("\n\n")

CURATOR: Liu
Title: Progression and Resolution of Severe Acute Respiratory Syndrome Coronavirus 2 Infection in Golden Syrian Hamsters
Journal: The American journal of pathology
Pubmed: https://pubmed.ncbi.nlm.nih.gov/34767812
PMID_34767812
Klein, Sabra L
CA260492



CURATOR: Liu
Title: Direct comparison of antibody responses to four SARS-CoV-2 vaccines in Mongolia
Journal: Cell host &amp; microbe
Pubmed: https://pubmed.ncbi.nlm.nih.gov/34861167
PMID_34861167
Boyd, Scott D
CA260517



